In [1]:
from thesis_v2.analysis.utils import LayerSourceAnalysis, get_source_analysis_for_one_model_spec
from collections import defaultdict

In [2]:
# 7 iteration network, `sources` gives the contribution at each time step,

# this network assumes that the bottom up input never changes,
# and the bottom up input is processed using conv layer `B1`
# and the lateral input is processed using conv layer `R1`
# at the end of time t (k=1,2,3,4,5,6,7), we apply a BN layer with (average) scale `s1,{t}`.
# bias, activation layers are not explictly represented here and will be ignored during analysis


sources = [LayerSourceAnalysis().add_source(conv=('I','B1',), scale=(1.0, 's1,1',))]

for t in range(2, 7+1):
    src_this = LayerSourceAnalysis().add_source(
        conv=('I','B1',), scale=(1.0,)
    )
    # input from recurrent
    src_prev_with_recurrent = sources[-1].apply_conv(('R1',))
    
    src_this = src_this.add(src_prev_with_recurrent)
    src_this = src_this.apply_scale((f's1,{t}',))
    
    sources.append(src_this)

In [3]:
[s.source_list for s in sources]
# this looks right, the same as my handwritten analysis.

[[{'conv': ('I', 'B1'), 'scale': (1.0, 's1,1')}],
 [{'conv': ('I', 'B1'), 'scale': (1.0, 's1,2')},
  {'conv': ('I', 'B1', 'R1'), 'scale': (1.0, 's1,1', 's1,2')}],
 [{'conv': ('I', 'B1'), 'scale': (1.0, 's1,3')},
  {'conv': ('I', 'B1', 'R1'), 'scale': (1.0, 's1,2', 's1,3')},
  {'conv': ('I', 'B1', 'R1', 'R1'), 'scale': (1.0, 's1,1', 's1,2', 's1,3')}],
 [{'conv': ('I', 'B1'), 'scale': (1.0, 's1,4')},
  {'conv': ('I', 'B1', 'R1'), 'scale': (1.0, 's1,3', 's1,4')},
  {'conv': ('I', 'B1', 'R1', 'R1'), 'scale': (1.0, 's1,2', 's1,3', 's1,4')},
  {'conv': ('I', 'B1', 'R1', 'R1', 'R1'),
   'scale': (1.0, 's1,1', 's1,2', 's1,3', 's1,4')}],
 [{'conv': ('I', 'B1'), 'scale': (1.0, 's1,5')},
  {'conv': ('I', 'B1', 'R1'), 'scale': (1.0, 's1,4', 's1,5')},
  {'conv': ('I', 'B1', 'R1', 'R1'), 'scale': (1.0, 's1,3', 's1,4', 's1,5')},
  {'conv': ('I', 'B1', 'R1', 'R1', 'R1'),
   'scale': (1.0, 's1,2', 's1,3', 's1,4', 's1,5')},
  {'conv': ('I', 'B1', 'R1', 'R1', 'R1', 'R1'),
   'scale': (1.0, 's1,1', 's1,2'

In [4]:
scale_map = {
    's1,1': 1.0,
    's1,2': 2.0,
    's1,3': 3.0,
    's1,4': 4.0,
    's1,5': 5.0,
    's1,6': 6.0,
    's1,7': 7.0,
}

In [5]:
[s.evaluate(scale_map) for s in sources]

[{('I', 'B1'): 1.0},
 {('I', 'B1'): 2.0, ('I', 'B1', 'R1'): 2.0},
 {('I', 'B1'): 3.0, ('I', 'B1', 'R1'): 6.0, ('I', 'B1', 'R1', 'R1'): 6.0},
 {('I', 'B1'): 4.0,
  ('I', 'B1', 'R1'): 12.0,
  ('I', 'B1', 'R1', 'R1'): 24.0,
  ('I', 'B1', 'R1', 'R1', 'R1'): 24.0},
 {('I', 'B1'): 5.0,
  ('I', 'B1', 'R1'): 20.0,
  ('I', 'B1', 'R1', 'R1'): 60.0,
  ('I', 'B1', 'R1', 'R1', 'R1'): 120.0,
  ('I', 'B1', 'R1', 'R1', 'R1', 'R1'): 120.0},
 {('I', 'B1'): 6.0,
  ('I', 'B1', 'R1'): 30.0,
  ('I', 'B1', 'R1', 'R1'): 120.0,
  ('I', 'B1', 'R1', 'R1', 'R1'): 360.0,
  ('I', 'B1', 'R1', 'R1', 'R1', 'R1'): 720.0,
  ('I', 'B1', 'R1', 'R1', 'R1', 'R1', 'R1'): 720.0},
 {('I', 'B1'): 7.0,
  ('I', 'B1', 'R1'): 42.0,
  ('I', 'B1', 'R1', 'R1'): 210.0,
  ('I', 'B1', 'R1', 'R1', 'R1'): 840.0,
  ('I', 'B1', 'R1', 'R1', 'R1', 'R1'): 2520.0,
  ('I', 'B1', 'R1', 'R1', 'R1', 'R1', 'R1'): 5040.0,
  ('I', 'B1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1'): 5040.0}]

In [6]:
# use my util function to achieve the same

In [7]:
sources_using_util_fn = get_source_analysis_for_one_model_spec(
    num_recurrent_layer=1, num_cls=7, readout_type=None, return_raw=True
)
assert len(sources_using_util_fn) == 2
sources_using_util_fn = sources_using_util_fn[-1]

In [8]:
# same
assert [s.source_list for s in sources_using_util_fn] == [s.source_list for s in sources]

In [9]:
(lambda: 1.0)()

1.0

In [10]:
# let's see different readout types

def check_different_readout_type():
    scale_map = {
        # a default dict won't work!!! because .get() in defaultdict is the same is regular dict.
        f's1,{t}': 1.0 for t in range(1, 7+1)
    }
    
    for readout_type in ('inst-last', 'cm-last', 'inst-avg', 'cm-avg'):
        src_this = get_source_analysis_for_one_model_spec(
            num_recurrent_layer=1, num_cls=7, readout_type=readout_type
        )
        print(readout_type)
        print(src_this.unique_convs)
        print(src_this.source_list)
        print(src_this.evaluate(scale_map))
        print(src_this.evaluate(scale_map, {'B1': 1.0, 'R1': 2.0, 'I': 1.0}))

In [11]:
check_different_readout_type()
# looks good!

inst-last
{('I', 'B1', 'R1', 'R1', 'R1'), ('I', 'B1'), ('I', 'B1', 'R1', 'R1'), ('I', 'B1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1'), ('I', 'B1', 'R1', 'R1', 'R1', 'R1', 'R1'), ('I', 'B1', 'R1', 'R1', 'R1', 'R1'), ('I', 'B1', 'R1')}
[{'conv': ('I', 'B1'), 'scale': (1.0, 's1,7')}, {'conv': ('I', 'B1', 'R1'), 'scale': (1.0, 's1,6', 's1,7')}, {'conv': ('I', 'B1', 'R1', 'R1'), 'scale': (1.0, 's1,5', 's1,6', 's1,7')}, {'conv': ('I', 'B1', 'R1', 'R1', 'R1'), 'scale': (1.0, 's1,4', 's1,5', 's1,6', 's1,7')}, {'conv': ('I', 'B1', 'R1', 'R1', 'R1', 'R1'), 'scale': (1.0, 's1,3', 's1,4', 's1,5', 's1,6', 's1,7')}, {'conv': ('I', 'B1', 'R1', 'R1', 'R1', 'R1', 'R1'), 'scale': (1.0, 's1,2', 's1,3', 's1,4', 's1,5', 's1,6', 's1,7')}, {'conv': ('I', 'B1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1'), 'scale': (1.0, 's1,1', 's1,2', 's1,3', 's1,4', 's1,5', 's1,6', 's1,7')}]
{('I', 'B1'): 1.0, ('I', 'B1', 'R1'): 1.0, ('I', 'B1', 'R1', 'R1'): 1.0, ('I', 'B1', 'R1', 'R1', 'R1'): 1.0, ('I', 'B1', 'R1', 'R1', 'R1', 'R1'): 1.